In [2]:
import numpy as np
import load_utils
import spine_augmentation as aug
import confidence_map as cmap
import ladder_shufflenet
import torch.optim as optim
import torch.nn as nn
import torch
import os.path as path
import torchvision
import matplotlib.pyplot as plt
import cv2
import torch.nn.functional as F
from PIL import Image
import folders as f
import os
import argparse


## Gaussian to point

Traverse all test images

In [25]:


run_on_submit_test(net)

01-July-2019-1
01-July-2019-10
01-July-2019-11
01-July-2019-12


KeyboardInterrupt: 

0 10.745457581700691
[ 1.5084058  -1.94397769  6.94378349]
1 19.505581095447607
[ -1.5091869   -6.05721344 -10.75297346]
2 25.99161922309971
[-4.91196055 12.44701481 12.57052464]
3 4.0159966221110395
[2.61067779 1.34372897 1.53394882]
4 16.933961157711614
[  2.80004315  18.53869197 -12.67894882]
5 19.82118632206795
[ -3.18643054  14.9860202  -33.05345073]
6 13.62765856438369
[0.02795464 3.67275107 1.65210357]
7 2.8786704819389777
[-1.37602277  2.67725468  1.78472255]
8 27.169516602857545
[-1.00420189  9.16234705  2.40185107]
9 14.833186440049372
[-6.72782107 -8.96875354 -3.08006753]
10 4.929105413110176
[-1.10448639  6.22656479 -0.95305118]
11 7.230083025630425
[-2.56380318  9.29010044 -2.52090362]
12 12.507501146623442
[ 1.48680927 -9.56819229  2.36900156]
13 28.10537993331108
[-1.005293   10.57833828 -8.78594828]
14 8.357694016052232
[  1.282       -4.70391976 -10.19608024]
15 18.321386007978766
[-0.60275295 -2.29739465  3.67294169]
16 6.750799290273641
[ 1.21549809  5.29354927 -4.92

In [27]:
print(np.mean(avg_smape))

14.711995701076575


validation set
compair with (1,0) straight line: 13.8  
compair with (1,0) straight line: 14.7 (delete some bones)
compair with most upper and lower: 20.21 (delete some bones)